# Workflow

## Step 1: Data extraction using Google Earth Engine (GEE)

The data extraction is performed using a script written in Google Earth Engine.

1. Open the following link in a web browser:  
   https://code.earthengine.google.com/c2a490d827f13217b3cc88ca2c7a0b3b

2. Click Run to execute the script.

3. After execution, open the Tasks tab.

4. Start the export task and download the generated CSV file to your local machine.

---

## Step 2: Python analysis in Jupyter Notebook / VS Code

The downloaded CSV file is used as input for the Python-based analysis.

### Install required dependencies

Install the required Python libraries based on the imports used in the notebook:


pip install pandas numpy matplotlib scikit-learn

## Step 3: Run the code

Now the code


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
 
# =========================
# 1) Load the CSV
# =========================
# Update filename if needed
df = pd.read_csv(r"C:\Users\numra\Downloads\python\co2.ipynb")
 
# Quick cleanup: make sure numeric cols are numeric
numeric_cols = [
    "EVI_season_mean",
    "NDVI_season_mean",
    "GPP_season_tCO2_ha",
    "Reco_season_tCO2_ha",
    "NEE_season_tCO2_ha",
    "months_with_S2",
    "year",
    "season_start_month",
    "season_end_month"
]
 
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")
 
# Drop rows with missing key values
df = df.dropna(subset=[
    "EVI_season_mean",
    "NDVI_season_mean",
    "GPP_season_tCO2_ha",
    "Reco_season_tCO2_ha",
    "year",
    "crop_key"
])
 
# =========================
# 2) Feature engineering
# =========================
# Season length and mid month (phenology differences)
df["season_length"] = df["season_end_month"] - df["season_start_month"] + 1
df["season_mid_month"] = (df["season_start_month"] + df["season_end_month"]) / 2.0
 
# NDVI-EVI gap
df["ndvi_minus_evi"] = df["NDVI_season_mean"] - df["EVI_season_mean"]
 
feature_cols = [
    "EVI_season_mean",
    "NDVI_season_mean",
    "ndvi_minus_evi",
    "months_with_S2",
    "year",
    "season_length",
    "season_mid_month",
    "crop_key"
]
 
X = df[feature_cols].copy()
 
# Targets
y_gpp = df["GPP_season_tCO2_ha"].copy()
y_reco = df["Reco_season_tCO2_ha"].copy()
 
# =========================
# 3) Preprocessing pipeline
# =========================
categorical_features = ["crop_key"]
numeric_features = [c for c in feature_cols if c not in categorical_features]
 
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(drop="first"), categorical_features),
    ]
)
 
# Ridge regression models
gpp_model = Pipeline(steps=[("prep", preprocess), ("model", Ridge(alpha=1.0))])
reco_model = Pipeline(steps=[("prep", preprocess), ("model", Ridge(alpha=1.0))])
 
# =========================
# 4) Validation: LOOCV
# =========================
loo = LeaveOneOut()
gpp_pred = cross_val_predict(gpp_model, X, y_gpp, cv=loo)
reco_pred = cross_val_predict(reco_model, X, y_reco, cv=loo)
 
# Metrics function
def metrics(y_true, y_hat):
    rmse = float(np.sqrt(mean_squared_error(y_true, y_hat)))
    mae = float(mean_absolute_error(y_true, y_hat))
    r2 = float(r2_score(y_true, y_hat))
    bias = float(np.mean(y_hat - y_true))
    return rmse, mae, r2, bias
 
rmse_gpp, mae_gpp, r2_gpp, bias_gpp = metrics(y_gpp, gpp_pred)
rmse_reco, mae_reco, r2_reco, bias_reco = metrics(y_reco, reco_pred)
 
print("=== LOOCV Performance ===")
print(f"GPP -> RMSE={rmse_gpp:.3f}, MAE={mae_gpp:.3f}, R2={r2_gpp:.3f}, Bias={bias_gpp:.3f}")
print(f"Reco -> RMSE={rmse_reco:.3f}, MAE={mae_reco:.3f}, R2={r2_reco:.3f}, Bias={bias_reco:.3f}")
 
# =========================
# 5) Compute NEE from predictions + compare
# =========================
nee_true = df["NEE_season_tCO2_ha"].values
nee_pred = gpp_pred - reco_pred  # consistent with your model definition
 
rmse_nee, mae_nee, r2_nee, bias_nee = metrics(nee_true, nee_pred)
print(f"NEE(pred=GPP_pred-Reco_pred) -> RMSE={rmse_nee:.3f}, MAE={mae_nee:.3f}, R2={r2_nee:.3f}, Bias={bias_nee:.3f}")
 
# =========================
# 6) Plot NEE by crop over time (Observed & Predicted)
# =========================
plot_df = df[["crop_key","crop_name","year","NEE_season_tCO2_ha"]].copy()
plot_df["NEE_pred"] = nee_pred
plot_df = plot_df.sort_values(["crop_key","year"])
 
# Observed NEE
plt.figure()
for crop in plot_df["crop_key"].unique():
    sub = plot_df[plot_df["crop_key"] == crop]
    plt.plot(sub["year"], sub["NEE_season_tCO2_ha"], marker="o", label=f"{crop} (obs)")
plt.axhline(0, linewidth=1)
plt.xlabel("Year")
plt.ylabel("NEE (tCO2/ha/season)")
plt.title("Seasonal Net CO2 (NEE) by crop — Observed")
plt.legend()
plt.show()
 
# Predicted NEE
plt.figure()
for crop in plot_df["crop_key"].unique():
    sub = plot_df[plot_df["crop_key"] == crop]
    plt.plot(sub["year"], sub["NEE_pred"], marker="o", label=f"{crop} (pred)")
plt.axhline(0, linewidth=1)
plt.xlabel("Year")
plt.ylabel("NEE (tCO2/ha/season)")
plt.title("Seasonal Net CO2 (NEE) by crop — Predicted from ML GPP & Reco")
plt.legend()
plt.show()
 
# =========================
# 7) Crop contribution summary
# =========================
summary = plot_df.groupby(["crop_key","crop_name"])["NEE_season_tCO2_ha"].agg(
    mean="mean",
    median="median",
    std="std",
    min="min",
    max="max",
    total="sum"
).reset_index()
 
print("\n=== Crop NEE summary (observed) ===")
print(summary.sort_values("mean"))
 
# Ranking for strongest sink (most uptake)
print("\nRanking (strongest sink / most uptake):")
print(summary.sort_values("mean", ascending=False)[["crop_key","crop_name","mean","total"]])
 
# Ranking for strongest source (most release)
print("\nRanking (strongest source / most release):")
print(summary.sort_values("mean", ascending=True)[["crop_key","crop_name","mean","total"]])
 
 

KeyError: 'EVI_season_mean'